In [20]:
from sympy import nextprime
import random

def generate_bit_prime(size):
    min_val = 2**size
    max_val = 2**size - 1
    
    start = random.randint(min_val, max_val)
    
    prime = nextprime(start)
    
    if prime > max_val:
        return generate_bit_prime(size)
    return prime

prime_20_bit: int = generate_bit_prime(20)
hex(prime_20_bit)


ValueError: empty range for randrange() (1048576, 1048576, 0)

In [ ]:
p = 0xD3915

F = GF(p)

In [ ]:

import random

def get_elliptic_parameters(size):
    # random r
    while True:
        random_bit = random.getrandbits(size)
        binary_rand = '0' + bin(random_bit)[2:].zfill(size)
        r = int(binary_rand, 2)

        # random B same manner as r
        random_bit = random.getrandbits(size)
        binary_rand = '0' + bin(random_bit)[2:].zfill(size)
        b = int(binary_rand, 2)

        R.<a> = PolynomialRing(F)
        L = (r*b**2-a**3).roots()

        for u in L:
            check = (4*u[0]**2 + 27*b**2) % p
            if check != 0:
                a = u[0]
                return a, b, r
    

a, b ,r = get_elliptic_parameters(20)
a, b, r

(738492, 694682, 926251)

### CURVE DEFINITION

In [4]:
## CURVE DEFINITION
p = 0xD3915
F = GF(p)
a = 738492
b = 694682
r = 926251
ECurv = EllipticCurve(F, [a, b])

print(f"Curve: {ECurv}")

Curve: Elliptic Curve defined by y^2 = x^3 + 738492*x + 694682 over Finite Field of size 866581


In [5]:
# Test case 1
p1 = ECurv(184224, 74658)
p2 = ECurv(428817, 567437)

p1+p2

(109605 : 690162 : 1)

In [7]:
# Test case 2
# p1 = ECurv(195914, 577386)
# p2 = ECurv(589027, 802679)

# p1+p2

In [9]:
# # Test case 3
# p1 = ECurv(195914, 577386)
# p2 = ECurv(589027, 802679)

# p1+p2

# ECurv

### POINT MULTIPLY TESTS

In [11]:
p1 = ECurv(264320, 549393)

p1

(632162 : 737833 : 1)

In [27]:
======= ECCp-20 =======
p = D3915

a = B44BC
b = A999A
r = E222B
h = 01
n = 865842

P_x = 74BF8
P_y = C557E

Q_x = 0679 834CEFB7 215DC365
Q_y = 4084 BC50388C 4E6FDFAB

SyntaxError: invalid decimal literal (819186979.py, line 10)